# Flexx demo - Hijacking the notebook

In [ ]:
from flexx import event, app, ui
app.init_notebook()

### Hijack the notebook

In [ ]:
class IPythonContainer(ui.Widget):
    
    @event.prop
    def hijack(self, v):
        return bool(v)
    
    class JS:
        
        @event.connect('hijack')
        def on_hijack(self, *events):
            hijack = events[-1].new_value
            for name in ('header', 'site'):
                el = document.getElementById(name)
                if hijack:
                    self.node.appendChild(el)
                else:
                    window.document.body.insertBefore(el, self.parent.node)

Create dockpanel and attach to `<body>`. We can't see it yet, because it it's "below" the notebook.

In [ ]:
dock = ui.DockPanel(container='body')
notebook = IPythonContainer(parent=dock, title='Notebook')

Now the hijack ....

In [ ]:
notebook.hijack = True

### Add some other widgets

In [ ]:
button = ui.Button(parent=dock, text='push me')

In [ ]:
button.title='Button!'

In [ ]:
from flexx.ui.examples.splines import Splines
with dock:
    splines = Splines(title='Splines')

In [ ]:
splines.closed.checked = True

### Add Bokeh plot

In [ ]:
from bokeh.models import Range1d, WMTSTileSource, ColumnDataSource, HoverTool
from bokeh.plotting import figure
from bokeh.sampledata.airports import data as airports

title = "US Airports: Field Elevation > 1500m"

points_source = ColumnDataSource(airports)
tile_source = WMTSTileSource(url='http://otile2.mqcdn.com/tiles/1.0.0/sat/{Z}/{X}/{Y}.png')

x_range = Range1d(start=airports['x'].min() - 10000, end=airports['x'].max() + 10000, bounds=None)
y_range = Range1d(start=airports['y'].min() - 10000, end=airports['y'].max() + 10000, bounds=None)

p = figure(tools='wheel_zoom,pan', x_range=x_range, y_range=y_range, title=title)
p.axis.visible = False
hover_tool = HoverTool(tooltips=[("Name", "@name"), ("Elevation", "@elevation (m)")])
p.add_tools(hover_tool)
p.add_tile(tile_source)
p.circle(x='x', y='y', size=9, fill_color="#60ACA1", line_color="#D2C4C1", line_width=1.5, source=points_source)

In [ ]:
bw = ui.BokehWidget(plot=p, title='Airports', style='height:400px')
bw

In [ ]:
bw.parent = dock

### Clean up

Undo the hijack, so that the notebook does not get messes up

In [ ]:
notebook.hijack = False